In [ ]:
# Importações
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# criar o navegador
options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install()) #Garante que o Chromedriver atualize automaticamente
nav = webdriver.Chrome(service = servico)

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

Definição das funções de busca no google e no buscape

In [ ]:

def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    # analise com os termos banidos
    
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos


def verificar_tem_todos_os_termos(lista_termos_nome_produto, nome):
    tem_todos_os_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_termos_banidos = False
    return tem_todos_os_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_de_ofertas = []
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)


    #Entrar no Google
    nav.get("https://www.google.com")
    nav.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(produto,Keys.ENTER)

    #Clicar na aba shopping
    elementos = nav.find_elements(By.CLASS_NAME,"hdtb-mitem")

    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break
    #Pegar informações dos produtos

    lista_resultados = nav.find_elements(By.CLASS_NAME,"i0X6df")

    for resultado in lista_resultados:

        nome = resultado.find_element(By.CLASS_NAME, "tAxDx").text
        nome = nome.lower()
        # Analise de termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        tem_todos_os_termos_produtos = verificar_tem_todos_os_termos(lista_termos_nome_produto, nome)


        #selecionar elementos que passaram nas duas analises
        if not tem_termos_banidos and tem_todos_os_termos_produtos:
            try:
                preco = resultado.find_element(By.CLASS_NAME, "a8Pemb").text
                preco = preco.replace("R$", "").replace(" ","").replace(".","").replace(",",".")
                preco = float(preco)
            except:
                continue

            #selecionar produtos que estão dentro da faixa de preços



            if preco_minimo <= preco <= preco_maximo:
                
                elemento_referencia = resultado.find_element(By.CLASS_NAME, "bONr3b")
                elemento_pai = elemento_referencia.find_element(By.XPATH,"..")
                link = elemento_pai.get_attribute("href")

                lista_de_ofertas.append((nome, preco, link))
    return lista_de_ofertas


def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # Tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_de_ofertas = []
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    
    # Buscar no buscapé
    nav.get("https://www.buscape.com.br")
    nav.find_element(By.XPATH, 
                     '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto,
                                                                                                              Keys.ENTER)
    
    # Pegar os resultados
    while len(nav.find_elements(By.CLASS_NAME,'Select_Select__1S7HV')) <1:
        time.sleep(1)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    for resultado in lista_resultados:
        
        preco = resultado.find_element(By.CLASS_NAME,'Text_MobileHeadingS__Zxam2').text
        
        nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        link = resultado.get_attribute("href")
        
    # Analisar o resultado
        # Analise de termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        tem_todos_os_termos_produtos = verificar_tem_todos_os_termos(lista_termos_nome_produto, nome)
    # Analisar se o preço está na faixa
        if not tem_termos_banidos and tem_todos_os_termos_produtos:
            try:
                preco = preco.replace("R$", "").replace(" ","").replace(".","").replace(",",".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_de_ofertas.append((nome, preco, link))
            except:
                continue
            
    # Retornar a lista de ofertas do buscapé
    return lista_de_ofertas

In [ ]:
#Compilando as ofertas em uma unica tabela

tabela_ofertas = pd.DataFrame()


for linha in tabela_produtos.index:
    #Itens da pesquisa
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
    lista_ofertas_busca_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_busca_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_busca_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
display(tabela_ofertas)

In [ ]:
#Exportar para o Excel
tabela_ofertas.to_excel("Ofertas.xlsx", index = False)


#Envio por email
import win32com.client as win32

if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ''
    mail.CC = ''
    mail.Subject = 'Segundo Desafio LIrado do Lira'
    mail.HTMLBody = f"""
    <p>Prezados, </p>
    <p>Encontrei esses produtos em oferta dentro da faixa desejada.</p>
    {tabela_ofertas.to_html(index = False)}
    <p>Cordialmente,</p>
    <p>Anderson H. S. Martins </p>
    """
    mail.Send()
    nav.quit()